To open this notebook in Colab visit https://goo.gl/magenta/groovae-colab

<img src="https://magenta-staging.tensorflow.org/assets/groovae/score-groove.png" alt="GrooVAE Figure" >

In [1]:
import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np

import tensorflow_datasets as tfds

import librosa
import note_seq
from note_seq.protobuf import music_pb2

from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

GROOVAE_2BAR_TAP_FIXED_VELOCITY="groovae_2bar_tap_fixed_velocity.tar"

/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present i

In [33]:
# If a sequence has notes at time before 0.0, scootch them up to 0
def start_notes_at_0(s):
    for n in s.notes:
        if n.start_time < 0:
            n.end_time -= n.start_time
            n.start_time = 0
    return s

# Some midi files come by default from different instrument channels
# Quick and dirty way to set midi files to be recognized as drums
def set_to_drums(ns):
    for n in ns.notes:
        n.instrument=9
        n.is_drum = True
        
# quickly change the tempo of a midi sequence and adjust all notes
def change_tempo(note_sequence, new_tempo):
    new_sequence = copy.deepcopy(note_sequence)
    ratio = note_sequence.tempos[0].qpm / new_tempo
    for note in new_sequence.notes:
        note.start_time = note.start_time * ratio
        note.end_time = note.end_time * ratio
    new_sequence.tempos[0].qpm = new_tempo
    return new_sequence

# Calculate quantization steps but do not remove microtiming
def quantize(s, steps_per_quarter=4):
    return note_seq.sequences_lib.quantize_note_sequence(s,steps_per_quarter)

# Destructively quantize a midi sequence
def flatten_quantization(s):
    beat_length = 60. / s.tempos[0].qpm
    step_length = beat_length / 4 #s.quantization_info.steps_per_quarter
    new_s = copy.deepcopy(s)
    for note in new_s.notes:
        note.start_time = step_length * note.quantized_start_step
        note.end_time = step_length * note.quantized_end_step
    return new_s

# Calculate how far off the beat a note is
def get_offset(s, note_index):
    q_s = flatten_quantization(quantize(s))
    true_onset = s.notes[note_index].start_time
    quantized_onset = q_s.notes[note_index].start_time
    diff = quantized_onset - true_onset
    beat_length = 60. / s.tempos[0].qpm
    step_length = beat_length / 4#q_s.quantization_info.steps_per_quarter
    offset = diff/step_length
    return offset

In [34]:
# Load MIDI files from GMD with MIDI only (no audio) as a tf.data.Dataset
dataset_2bar = tfds.as_numpy(tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.VALIDATION,
    try_gcs=True))

dev_sequences = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]
print(len(dev_sequences))

2039


In [146]:
for features in dataset_2bar:
    x=note_seq.midi_to_note_sequence(features["midi"])
    break

In [147]:
x

ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 180.00018000018
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.14652763125
  end_time: 0.247221975
  is_drum: true
}
notes {
  pitch: 55
  velocity: 127
  start_time: 0.3402774375
  end_time: 0.44027733750000003
  is_drum: true
}
notes {
  pitch: 38
  velocity: 127
  start_time: 0.34305521250000004
  end_time: 0.44374955625
  is_drum: true
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.6631937812500001
  end_time: 0.7645825687500001
  is_drum: true
}
notes {
  pitch: 26
  velocity: 127
  start_time: 0.6819437625
  end_time: 0.7826381062500001
  is_drum: true
}
notes {
  pitch: 40
  velocity: 127
  start_time: 0.9861101250000001
  end_time: 1.0874989125
  is_drum: true
}
notes {
  pitch: 26
  velocity: 127
  start_time: 0.9951378937500001
  end_time: 1.0972211250000001
  is_drum: true
}
notes {
  pitch: 36
  velocity: 127
  start_time: 1.16110995
  end_time: 1.2624

In [148]:
quantize(x)

ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 180.00018000018
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.14652763125
  end_time: 0.247221975
  is_drum: true
  quantized_start_step: 2
  quantized_end_step: 3
}
notes {
  pitch: 55
  velocity: 127
  start_time: 0.3402774375
  end_time: 0.44027733750000003
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
  pitch: 38
  velocity: 127
  start_time: 0.34305521250000004
  end_time: 0.44374955625
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.6631937812500001
  end_time: 0.7645825687500001
  is_drum: true
  quantized_start_step: 8
  quantized_end_step: 9
}
notes {
  pitch: 26
  velocity: 127
  start_time: 0.6819437625
  end_time: 0.7826381062500001
  is_drum: true
  quantized_start_step: 8
  quantized_end_step: 9
}
notes {
  pitch: 40
  velocity: 127
  start_time: 0.9861101

In [56]:
dev_sequences[0]

ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 180.00018000018
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.14652763125
  end_time: 0.247221975
  instrument: 9
  is_drum: true
  quantized_start_step: 2
  quantized_end_step: 3
}
notes {
  pitch: 55
  velocity: 127
  start_time: 0.3402774375
  end_time: 0.44027733750000003
  instrument: 9
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
  pitch: 38
  velocity: 127
  start_time: 0.34305521250000004
  end_time: 0.44374955625
  instrument: 9
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
  pitch: 36
  velocity: 127
  start_time: 0.6631937812500001
  end_time: 0.7645825687500001
  instrument: 9
  is_drum: true
  quantized_start_step: 8
  quantized_end_step: 9
}
notes {
  pitch: 26
  velocity: 127
  start_time: 0.6819437625
  end_time: 0.7826381062500001
  instrument: 9
  is_drum: true
  quantized_start_step: 8
  qua

# Tap2Drum: Generate a beat from any rhythm 

While the Groove model works by removing the micro-timing and velocity information and learning to predict them from just the drum pattern, we can also go in the opposite direction.  Here, we take a representation of a Groove as input (in the form of a rhythm that can have precise timing but where drum categories are ignored) - and then generate drum beats that match the groove implied by this rhythm.  We trained this model by collapsing all drum hits from each beat in the training data to a single "tapped" rhythm, and then learning to decode full beats from that rhythm.  This allows us to input any rhythm we like through the precise onset timings in a "tap" and let the model decode our rhythm into a beat. We can even simply record taps as audio, or extract them from a recording of another instrument, rather than needing a midi controller.

In [69]:
# Load some configs to be used later
dc_tap = configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'].data_converter

# TODO: Control temperature with OSC

In [35]:
# quick method for turning a drumbeat into a tapped rhythm
def get_tapped_2bar(s, velocity=85, ride=False):
    new_s = dc_tap.from_tensors(dc_tap.to_tensors(s).inputs)[0]
    new_s = change_tempo(new_s, s.tempos[0].qpm)
    if velocity != 0:
        for n in new_s.notes:
            n.velocity = velocity
    if ride:
        for n in new_s.notes:
            n.pitch = 42
    return new_s

def drumify(s, model, temperature=1.0): 
    encoding, mu, sigma = model.encode([s])
    decoded = model.decode(encoding, length=32, temperature=temperature)
    return decoded[0]

def make_tap_sequence(tempo, onset_times, onset_frames, onset_velocities,
                     velocity_threshold, start_time, end_time):
    note_sequence = music_pb2.NoteSequence()
    note_sequence.tempos.add(qpm=tempo)
    for onset_vel, onset_time in zip(onset_velocities, onset_times):
        if onset_vel > velocity_threshold and onset_time >= start_time and onset_time < end_time:  # filter quietest notes
            note_sequence.notes.add(
            instrument=9, pitch=42, is_drum=True,
            velocity=onset_vel,  # model will use fixed velocity here
            start_time=onset_time - start_time,
            end_time=onset_time -start_time + 0.01
            )
    return note_sequence

Here are a couple of examples using MIDI rhythms:

NoteSequence => get_tapped_2bar => drumify

In [37]:
groovae_2bar_tap = TrainedModel(config=configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'],
                                batch_size=1,
                                checkpoint_dir_or_path=GROOVAE_2BAR_TAP_FIXED_VELOCITY)

sequence_indices = [1111, 366]
for i in sequence_indices:
    s = start_notes_at_0(dev_sequences[i])
    s = change_tempo(get_tapped_2bar(s, velocity=85, ride=True), dev_sequences[i].tempos[0].qpm)
    h = change_tempo(drumify(s, groovae_2bar_tap), s.tempos[0].qpm)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]



INFO:tensorflow:
Decoder Cells:
  units: [256, 256]



...

/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
Instructions for updating:
Do not call `graph_parents`.


INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Unbundling checkpoint.


INFO:tensorflow:Restoring parameters from /tmp/tmpcfnfmejf/groovae_2bar_tap_fixed_velocity/model.ckpt-3668


INFO:tensorflow:Restoring parameters from /tmp/tmpcfnfmejf/groovae_2bar_tap_fixed_velocity/model.ckpt-3668


# IO Communication

quantized_start_step, quantized_end_step  
control_changes {
  control_number: 4
  control_value: 90
  is_drum: true
}

In [162]:
def max_list_to_midi_array(max_list, BPM):
    beat_dur=60/BPM # in secs
    midi_array=[]
    for i in range((len(max_list)//3)):
        start_step=float(max_list[3*i]) # in beats
        end_step=float(max_list[3*i+1]) # in beats
        vel=float(max_list[3*i+2])
        start_time=start_step*beat_dur
        end_time=end_step*beat_dur
        midi_array.append([start_time,end_time,vel])
    return np.array(midi_array)

def make_tap_sequence_(tempo, midi_array, tpq=220):
    note_sequence=music_pb2.NoteSequence()
    note_sequence.tempos.add(qpm=tempo)
    note_sequence.ticks_per_quarter=tpq
    note_sequence.time_signatures.add(numerator=4, denominator=4)
    note_sequence.key_signatures.add()
    for onset_time, offset_time, onset_velocity in midi_array:
        if onset_velocity:
            note_sequence.notes.add(instrument=9, # Drum MIDI Program number
                                    pitch=42, # Constant
                                    is_drum=True,
                                    velocity=int(onset_velocity),
                                    start_time=onset_time,
                                    end_time=offset_time)
    note_sequence.total_time=2*4*60/BPM # 2bars
    return note_sequence  

In [154]:
slt="0. 0.03 120. 0.0625 0.0925 120. 0.125 0.155 0. 0.1875 0.2175 0. 0.25 0.28 120. 0.3125 0.3425 0. 0.375 0.405 0. 0.4375 0.4675 0. 0.5 0.53 0. 0.5625 0.5925 0. 0.625 0.655 0. 0.6875 0.7175 120. 0.75 0.78 0. 0.8125 0.8425 0. 0.875 0.905 120. 0.9375 0.9675 0. 1. 1.03 120. 1.0625 1.0925 120. 1.125 1.155 0. 1.1875 1.2175 0. 1.25 1.28 0. 1.3125 1.3425 0. 1.375 1.405 0. 1.4375 1.4675 0. 1.5 1.53 0. 1.5625 1.5925 0. 1.625 1.655 0. 1.6875 1.7175 0. 1.75 1.78 0. 1.8125 1.8425 0. 1.875 1.905 0. 1.9375 1.9675 0.".split(' ')

In [163]:
BPM=60
midi_array=max_list_to_midi_array(slt, BPM)
len(midi_array)

note_sequence=quantize(make_tap_sequence_(BPM, midi_array))
#note_sequence=start_notes_at_0(note_sequence)
#s=get_tapped_2bar(note_sequence, velocity=85, ride=True)
dc_tap.to_tensors(note_sequence)

ConverterTensors(inputs=[], outputs=[], controls=[], lengths=[])

In [160]:
note_sequence

ticks_per_quarter: 960
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 60.0
}
notes {
  pitch: 42
  velocity: 120
  end_time: 0.03
  instrument: 9
  is_drum: true
  quantized_end_step: 1
}
notes {
  pitch: 42
  velocity: 120
  start_time: 0.0625
  end_time: 0.0925
  instrument: 9
  is_drum: true
  quantized_end_step: 1
}
notes {
  pitch: 42
  velocity: 120
  start_time: 0.25
  end_time: 0.28
  instrument: 9
  is_drum: true
  quantized_start_step: 1
  quantized_end_step: 2
}
notes {
  pitch: 42
  velocity: 120
  start_time: 0.6875
  end_time: 0.7175
  instrument: 9
  is_drum: true
  quantized_start_step: 3
  quantized_end_step: 4
}
notes {
  pitch: 42
  velocity: 120
  start_time: 0.875
  end_time: 0.905
  instrument: 9
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
  pitch: 42
  velocity: 120
  start_time: 1.0
  end_time: 1.03
  instrument: 9
  is_drum: true
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
 

In [165]:
dc_tap.to_tensors(dev_sequences[i])

ConverterTensors(inputs=[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.15      ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.     

groovae_2bar_tap = TrainedModel(config=configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'],
                                batch_size=1,
                                checkpoint_dir_or_path=GROOVAE_2BAR_TAP_FIXED_VELOCITY)

sequence_indices = [1111, 366]
for i in sequence_indices:
    s = start_notes_at_0(dev_sequences[i])
    s = change_tempo(get_tapped_2bar(s, velocity=85, ride=True), dev_sequences[i].tempos[0].qpm)
    h = change_tempo(drumify(s, groovae_2bar_tap), s.tempos[0].qpm)

In [52]:
midi_array

array([[0.0000e+00, 1.5000e-02, 1.2000e+02],
       [3.1250e-02, 4.6250e-02, 0.0000e+00],
       [6.2500e-02, 7.7500e-02, 0.0000e+00],
       [9.3750e-02, 1.0875e-01, 1.2000e+02],
       [1.2500e-01, 1.4000e-01, 0.0000e+00],
       [1.5625e-01, 1.7125e-01, 1.2000e+02],
       [1.8750e-01, 2.0250e-01, 0.0000e+00],
       [2.1875e-01, 2.3375e-01, 0.0000e+00],
       [2.5000e-01, 2.6500e-01, 0.0000e+00],
       [2.8125e-01, 2.9625e-01, 0.0000e+00],
       [3.1250e-01, 3.2750e-01, 0.0000e+00],
       [3.4375e-01, 3.5875e-01, 0.0000e+00],
       [3.7500e-01, 3.9000e-01, 0.0000e+00],
       [4.0625e-01, 4.2125e-01, 0.0000e+00],
       [4.3750e-01, 4.5250e-01, 0.0000e+00],
       [4.6875e-01, 4.8375e-01, 0.0000e+00],
       [5.0000e-01, 5.1500e-01, 1.2000e+02],
       [5.3125e-01, 5.4625e-01, 0.0000e+00],
       [5.6250e-01, 5.7750e-01, 0.0000e+00],
       [5.9375e-01, 6.0875e-01, 0.0000e+00],
       [6.2500e-01, 6.4000e-01, 0.0000e+00],
       [6.5625e-01, 6.7125e-01, 0.0000e+00],
       [6.

In [64]:
note_sequence

tempos {
  qpm: 120.0
}
notes {
  pitch: 42
  velocity: 120
  end_time: 0.015
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 120
  start_time: 0.09375
  end_time: 0.10875
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 120
  start_time: 0.15625
  end_time: 0.17125
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 120
  start_time: 0.5
  end_time: 0.515
  instrument: 9
  is_drum: true
}

s = start_notes_at_0(dev_sequences[i])
s = change_tempo(get_tapped_2bar(s, velocity=85, ride=True), dev_sequences[i].tempos[0].qpm)
h = change_tempo(drumify(s, groovae_2bar_tap), s.tempos[0].qpm)

#get_tapped_2bar(note_sequence)
dc_tap.to_tensors(note_sequence)

drumify(note_sequence, groovae_2bar_tap)

In [65]:
s = start_notes_at_0(note_sequence)
s = change_tempo(get_tapped_2bar(s, velocity=85, ride=True), note_sequence.tempos[0].qpm)
h = change_tempo(drumify(s, groovae_2bar_tap), s.tempos[0].qpm)

IndexError: list index out of range

In [66]:
s = start_notes_at_0(note_sequence)

# Create pretty_midi

In [166]:
import pretty_midi

In [ ]:
cello_c_chord = pretty_midi.PrettyMIDI()
# Create an Instrument instance for a cello instrument
cello_program = pretty_midi.instrument_name_to_program('Cello')
cello = pretty_midi.Instrument(program=cello_program)
# Iterate over note names, which will be converted to note number later
for note_name in ['C5', 'E5', 'G5']:
    # Retrieve the MIDI note number for this note name
    note_number = pretty_midi.note_name_to_number(note_name)
    # Create a Note instance, starting at 0s and ending at .5s
    note = pretty_midi.Note(
        velocity=100, pitch=note_number, start=0, end=.5)
    # Add it to our cello instrument
    cello.notes.append(note)
# Add the cello instrument to the PrettyMIDI object
cello_c_chord.instruments.append(cello)

In [173]:
midi=pretty_midi.PrettyMIDI()
cello_program = pretty_midi.instrument_name_to_program('Cello')
cello = pretty_midi.Instrument(program=cello_program)
for onset_time, offset_time, onset_velocity in midi_array:
    cello.notes.append(
        pretty_midi.Note(
            velocity=85,
            pitch=42,
            start=onset_time,
            end=offset_time
    ))
midi.instruments.append(cello)

dev_sequences = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]
print(len(dev_sequences))

In [180]:
note_sequence=note_seq.midi_to_note_sequence(midi)
q_note_sequence=quantize(note_sequence)
set_to_drums(q_note_sequence)
dc_tap.to_tensors(q_note_sequence)

ConverterTensors(inputs=[array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 